In [37]:
import os
import numpy as np
import pandas as pd
import json

#raw data folder name
_rawData = '/rawData/'
#labeled dataset folder name 
_labeledData = '/labeledData/'

# Path to data
dataPath = os.getcwd()+'/dataset/'
rawDataPath = dataPath + _rawData
labeledDataPath = dataPath + _labeledData
saveDatesetPath = dataPath + '/output/'


# Extract and syncronize data 
it extact data from each trial and save it to 
    /dataset/labeledData/

In [15]:
import numpy as np
import pandas as pd
import os 

class make_folder_dataset:
    def __init__(self, folder_path:str) -> None:
        self.path = folder_path
        self.save_path = folder_path.replace(_rawData, _labeledData )
        os.makedirs(self.save_path)
        self.num_lines_per_message = 130
        self.df = pd.DataFrame()
        self.df_dataset = pd.DataFrame()
        self.tau = ['tau_J0','tau_J1', 'tau_J2', 'tau_J3', 'tau_J4', 'tau_J5', 'tau_J6']
        self.tau_d = ['tau_J_d0','tau_J_d1', 'tau_J_d2', 'tau_J_d3', 'tau_J_d4', 'tau_J_d5', 'tau_J_d6']
        self.tau_ext =['tau_ext0','tau_ext1','tau_ext2','tau_ext3','tau_ext4','tau_ext5','tau_ext6']

        self.q = ['q0','q1','q2','q3','q4','q5','q6']
        self.q_d = ['q_d0','q_d1','q_d2','q_d3','q_d4','q_d5','q_d6']

        self.dq = ['dq0','dq1','dq2','dq3','dq4','dq5','dq6']
        self.dq_d = ['dq_d0','dq_d1','dq_d2','dq_d3','dq_d4','dq_d5','dq_d6']


        self.e = ['e0','e1','e2','e3','e4','e5','e6']
        self.de = ['de0','de1','de2','de3','de4','de5','de6']
        self.etau = ['etau_J0','etau_J1', 'etau_J2', 'etau_J3', 'etau_J4', 'etau_J5', 'etau_J6']
    
    def _extract_array(self, data_dict:dict, data_frame:str, header:list,  n:int):
            dof = 7
            x, y = data_frame[n].split(':')
            y = y.replace('[','')
            y = y.replace(']','')
            y = y.replace('\n','')

            y = y.split(',')
            for i in range(dof):
                data_dict[header[i]].append(float(y[i]))

    def extract_robot_data(self):
        # it extracts robot data from all_data.txt
        f = open(self.path + 'all_data.txt', 'r')
        lines = f.readlines()

        keywords = ['time'] + self.tau + self.tau_d + self.tau_ext + self.q + self.q_d + self.dq + self.dq_d 

        data_dict = dict.fromkeys(keywords)
        for i in keywords:
            data_dict[i]=[0]
        
        for i in range(int(len(lines)/self.num_lines_per_message)):
            data_frame = lines[i*self.num_lines_per_message:(i+1)*self.num_lines_per_message]
            
            x, y = data_frame[3].split(':')
            time_ = int(y)-int(int(y)/1000000)*1000000

            x, y = data_frame[4].split(':')
            time_ = time_+int(y)/np.power(10,9)

            data_dict['time'].append(time_)
            
            self._extract_array(data_dict,data_frame,self.tau, 25)
            self._extract_array(data_dict,data_frame,self.tau_d, 26)
            self._extract_array(data_dict,data_frame, self.tau_ext, 37)
            
            self._extract_array(data_dict,data_frame,self.q, 28)
            
            self._extract_array(data_dict,data_frame, self.q_d, 29)
            self._extract_array(data_dict,data_frame, self.dq, 30)
            self._extract_array(data_dict,data_frame, self.dq_d, 31)
        
       
        self.df = pd.DataFrame.from_dict(data_dict)
        self.df = self.df.drop(index=0).reset_index()
        
        for i in range(len(self.e)):
            self.df[self.e[i]] = self.df[self.q_d[i]]-self.df[self.q[i]]
        for i in range(len(self.de)):
            self.df[self.de[i]] = self.df[self.dq_d[i]]-self.df[self.dq[i]]
        for i in range(len(self.etau)):
            self.df[self.etau[i]] = self.df[self.tau_d[i]]-self.df[self.tau[i]]

        #self.df.to_csv(self.save_path +'robot_data.csv',index=False)

    def get_labels(self):
        # it syncronize true labeled (contact- noncontact) data with robot data
        true_label = pd.read_csv(self.path+'true_label.csv')
        true_label['time'] = true_label['time_sec']+true_label['time_nsec']-self.df['time'][0]
        time_dev = true_label['time'].diff()
        contact_events_index = np.append([0], true_label['time'][time_dev>0.05].index.values)
        contact_events_index = np.append(contact_events_index,  true_label['time'].shape[0]-1)

        self.df['time'] = self.df['time'] - self.df['time'][0]
        contact_count = 0
        self.df['label']=0

        for i in range(self.df['time'].shape[0]):
            if (self.df['time'][i]-true_label['time'][contact_events_index[contact_count]]) > 0:
                #print(i ,',', contact_events_index[contact_count], ',',self.df['time'][i], '   ', true_label['time'][contact_events_index[contact_count]] )
                contact_count += 1
                if contact_count == len(contact_events_index):
                    break
                for j in range(i, self.df['time'].shape[0]):
                    self.df.loc[j, 'label'] = 1
                    #print(j)
                    if (self.df['time'][j] - true_label['time'][contact_events_index[contact_count]-1]) > 0:
                        #print(j ,',', contact_events_index[contact_count]-1, ',', self.df['time'][j], '   ', true_label['time'][contact_events_index[contact_count]-1] )
                        #print('----------------------------------------')
                        i = j
                        break

        self.df.to_csv(self.save_path + 'labeled_data.csv', index=False)
        
    def make_sequence(self):
        #window_time = 140ms
        seq_num = 28
        gap = 4
        selected_features= self.e + self.tau

        dataset = pd.DataFrame(np.ones((int((self.df.shape[0]-seq_num)/gap), seq_num*len(selected_features)+1))*2 )
        index = 0
        state = False
        last_contact_indexes = self.df.loc[self.df['label']==1,'index'].values
        last_contact_indexes = last_contact_indexes[last_contact_indexes.shape[0]-1]

        for i in range(0, last_contact_indexes, gap):
            if state: 
                window = self.df[selected_features][i:i+seq_num]
                dataset.iloc[index,0] = self.df['label'][i+seq_num]
                dataset.iloc[index, 1:len(dataset.columns)] = np.hstack(window.to_numpy())
                index += 1
            else:
                if self.df['label'][i+seq_num] == 1:
                    state = 1
        self.dataset = dataset.drop(index=dataset.loc[dataset[0]==2,0].index)

        name = self.path.split('/')[len(self.path.split('/'))-2]+'.csv'
        self.dataset.to_csv(self.save_path+name, index=False)
        return self.dataset
    
    def split_data(self, train_split_rate = 0.75):
        msk = np.random.rand(len(self.dataset)) < train_split_rate
        train = self.dataset.loc[msk, :]
        test = self.dataset.loc[~msk, :]
        name = self.path.split('/')[len(self.path.split('/'))-2]+'_train.csv'
        train.to_csv(self.save_path+name, index=False)

        name = self.path.split('/')[len(self.path.split('/'))-2]+'_test.csv'
        test.to_csv(self.save_path+name, index=False)



In [16]:

for i in os.listdir(rawDataPath):
    if len(i.split('.'))==1:
        submain_path = rawDataPath+i+'/'
        print(submain_path)
        for j in os.listdir(submain_path):
            if len(j.split('.'))==1:
                instance = make_folder_dataset(submain_path+j+'/')
                instance.extract_robot_data()
                instance.get_labels()
                
instance.df.head()

/home/mindlab/myProjects/robotHumanObjectDetectionDataset/dataset/rawData/configuration1/
/home/mindlab/myProjects/robotHumanObjectDetectionDataset/dataset/rawData/in_motion/


,index,time,tau_J0,tau_J1,tau_J2,tau_J3,tau_J4,tau_J5,tau_J6,tau_J_d0,...,de5,de6,etau_J0,etau_J1,etau_J2,etau_J3,etau_J4,etau_J5,etau_J6,label
0,1,0.000000,-0.045922,-10.235437,1.564731,21.741060,1.070547,2.436527,-0.002170,0.0,...,0.001677,0.011245,0.045922,10.235437,-1.564731,-21.741060,-1.070547,-2.436527,0.002170,0
1,2,0.004796,0.362708,-10.485376,1.814670,21.594271,1.028432,2.442935,0.018887,0.0,...,0.002799,0.007320,-0.362708,10.485376,-1.814670,-21.594271,-1.028432,-2.442935,-0.018887,0
2,3,0.009796,0.362708,-10.485376,1.814670,21.594271,1.028432,2.442935,0.018887,0.0,...,0.002799,0.007320,-0.362708,10.485376,-1.814670,-21.594271,-1.028432,-2.442935,-0.018887,0
3,4,0.014771,0.414283,-10.433802,1.814670,21.792635,1.034841,2.469486,0.018887,0.0,...,-0.002822,0.006148,-0.414283,10.433802,-1.814670,-21.792635,-1.034841,-2.469486,-0.018887,0
4,5,0.019798,0.414283,-10.433802,1.814670,21.792635,1.034841,2.469486,0.018887,0.0,...,-0.002822,0.006148,-0.414283,10.433802,-1.814670,-21.792635,-1.034841,-2.469486,-0.018887,0


# plot data

In [ ]:
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [35]:
target = ['e1','tau_J1']

for i in target:
    A = instance.df[i].max()-instance.df[i].min()
    instance.df['label_scaled']=instance.df['label']*A + instance.df[i][0] -A/2
    instance.df.iplot(x='time', y= [i, 'label_scaled'], xTitle='time (sec)', yTitle=i)
    
    #plt.plot(instance.df['time'],instance.df['labeled_scaled'])
    

In [ ]:
df_master_master_train[0][df_master_master_train[0]==1]
target = ['e0','tau_J0']

for i in target:
    A = instance.df[i].max()-instance.df[i].min()
    instance.df['label_scaled']=instance.df['label']*A + instance.df[i][0] -A/2
    instance.df.iplot(x='time', y= [i, 'label_scaled'], xTitle='time (sec)', yTitle)

In [ ]:
df_master_master_train[0][df_master_master_train[0]==1]

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,0.0,-0.478357,-47.147060,-1.168729,20.947603,0.802297,2.320255,-0.275913,-0.480253,-0.380135,...,0.000033,0.000012,0.000012,0.000127,-0.000708,-0.000172,0.000106,0.000030,-0.000406,0.000497
1,0.0,-0.478357,-47.218472,-1.168729,20.975374,0.820608,2.326663,-0.296054,-0.473540,-0.382516,...,0.000033,0.000013,0.000016,0.000100,0.000677,0.000410,-0.000951,0.000087,-0.000176,0.001887
2,0.0,-0.478357,-47.170864,-1.168729,20.947603,0.814199,2.333072,-0.296054,-0.475101,-0.388154,...,0.000033,0.000013,0.000003,-0.000390,0.000116,0.000212,0.000250,-0.000260,-0.000431,-0.001831
3,0.0,-0.478357,-47.170864,-1.192532,20.975374,0.814199,2.320255,-0.303379,-0.485631,-0.389599,...,0.000032,0.000013,0.000011,0.000542,-0.000266,0.000239,-0.000759,-0.000425,0.000105,0.001061
4,0.0,-0.478357,-47.218472,-1.140958,20.947603,0.820608,2.326663,-0.289646,-0.476251,-0.383315,...,0.000033,0.000013,0.000015,0.001261,-0.000312,0.000247,0.000727,0.000052,-0.000212,0.001547


# Make Dataset


## Features

In [17]:
tau = ['tau_J0','tau_J1', 'tau_J2', 'tau_J3', 'tau_J4', 'tau_J5', 'tau_J6']
tau_d = ['tau_J_d0','tau_J_d1', 'tau_J_d2', 'tau_J_d3', 'tau_J_d4', 'tau_J_d5', 'tau_J_d6']
tau_ext =['tau_ext0','tau_ext1','tau_ext2','tau_ext3','tau_ext4','tau_ext5','tau_ext6']

q = ['q0','q1','q2','q3','q4','q5','q6']
q_d = ['q_d0','q_d1','q_d2','q_d3','q_d4','q_d5','q_d6']

dq = ['dq0','dq1','dq2','dq3','dq4','dq5','dq6']
dq_d = ['dq_d0','dq_d1','dq_d2','dq_d3','dq_d4','dq_d5','dq_d6']


e = ['e0','e1','e2','e3','e4','e5','e6']
de = ['de0','de1','de2','de3','de4','de5','de6']
etau = ['etau_J0','etau_J1', 'etau_J2', 'etau_J3', 'etau_J4', 'etau_J5', 'etau_J6']

## required functions

In [18]:
def split_data(dataset, train_split_rate = 0.75):
    msk = np.random.rand(len(dataset)) < train_split_rate
    train = dataset.loc[msk, :]
    test = dataset.loc[~msk, :]
    return train, test

def make_sequence(df, selected_features, seq_num = 28, gap = 4):
    dataset = pd.DataFrame(np.ones((int((df.shape[0]-seq_num)/gap), seq_num*len(selected_features)+1))*2 )
    index = 0
    state = False
    last_contact_indexes = df.loc[df['label']==1,'index'].values
    last_contact_indexes = last_contact_indexes[last_contact_indexes.shape[0]-1]

    for i in range(0, last_contact_indexes, gap):
        if state: 
            window =df[selected_features][i:i+seq_num]
            dataset.iloc[index,0] = df['label'][i+seq_num]
            dataset.iloc[index, 1:len(dataset.columns)] = np.hstack(window.to_numpy())
            index += 1
        else:
            if df['label'][i+seq_num] == 1:
                state = 1
    dataset = dataset.drop(index=dataset.loc[dataset[0]==2,0].index)
    return dataset

def make_sequence_full_time(df, selected_features, seq_num = 28, gap = 4):
    dataset = pd.DataFrame(np.ones((int((df.shape[0]-seq_num)), seq_num*len(selected_features)+1))*2 )
    index = 0
    last_contact_indexes = df.loc[df['label']==1,'index'].values
    last_contact_indexes = last_contact_indexes[last_contact_indexes.shape[0]-1]

    for i in range(0, df.shape[0]-seq_num-1, gap):

        window =df[selected_features][i:i+seq_num]
        dataset.iloc[index,0] = df['label'][i+seq_num]
        dataset.iloc[index, 1:len(dataset.columns)] = np.hstack(window.to_numpy())
        index += 1

    dataset = dataset.drop(index=dataset.loc[dataset[0]==2,0].index)
    return dataset

## make a dataset with a set of features


In [21]:
os.makedirs(saveDatesetPath)
train_split_rate = 0.7

selected_features = tau + tau_ext + e + de
seq_num = 28

dict_label = {'a': 7, 'b':6, 'c':5, 'd':4, 'e':3, 'f':2, 'g':1}
columns = range(seq_num*len(selected_features)+1)

df_master_master_train = pd.DataFrame(columns=columns)
df_master_master_test = pd.DataFrame(columns=columns)


    
for i in os.listdir(labeledDataPath):
    if len(i.split('.'))==1:

        df_master_train = pd.DataFrame(columns=columns)
        df_master_test = pd.DataFrame(columns=columns)

        submain_path = labeledDataPath+i+'/'
        print(submain_path)
        for tag_name in os.listdir(submain_path):
            if len(tag_name.split('.'))==1:
                file_path = submain_path+tag_name+'/labeled_data.csv'
                df = pd.read_csv(file_path)
                df = make_sequence_full_time(df, selected_features,seq_num)
                #labeling data
                df[0] = df[0]*dict_label[tag_name[0]]
                train, test = split_data(df, train_split_rate)
                df_master_train = df_master_train.append(train, ignore_index=True)
                df_master_test = df_master_test.append(test, ignore_index=True)
                
        df_master_train.to_pickle(saveDatesetPath+i+'_train.pkl')
        df_master_test.to_pickle(saveDatesetPath+i+'_test.pkl')
        
    df_master_master_train = df_master_master_train.append(df_master_train, ignore_index=True)
    df_master_master_test = df_master_master_test.append(df_master_test, ignore_index=True)

df_master_master_train.to_pickle(saveDatesetPath+'dataset_train.pkl')
df_master_master_test.to_pickle(saveDatesetPath+'dataset_test.pkl')


/home/mindlab/myProjects/robotHumanObjectDetectionDataset/dataset//labeledData/configuration1/
/home/mindlab/myProjects/robotHumanObjectDetectionDataset/dataset//labeledData/in_motion/


# # Print Dataset Information

In [22]:
print('number of all samples: ', df_master_master_train.shape[0]+df_master_master_test.shape[0])
print('number of noncontact samples: ', df_master_master_train[0][df_master_master_train[0]==0].shape[0] +  df_master_master_test[0][df_master_master_test[0]==0].shape[0] )
print('number of contact samples: ', df_master_master_train[0][df_master_master_train[0]!=0].shape[0] +  df_master_master_test[0][df_master_master_test[0]!=0].shape[0] )
print('Link1 samples: ', df_master_master_train[0][df_master_master_train[0]==1].shape[0] +  df_master_master_test[0][df_master_master_test[0]==1].shape[0] )
print('Link2 samples: ', df_master_master_train[0][df_master_master_train[0]==2].shape[0] +  df_master_master_test[0][df_master_master_test[0]==2].shape[0] )
print('Link3 samples: ', df_master_master_train[0][df_master_master_train[0]==3].shape[0] +  df_master_master_test[0][df_master_master_test[0]==3].shape[0] )
print('Link4 samples: ', df_master_master_train[0][df_master_master_train[0]==4].shape[0] +  df_master_master_test[0][df_master_master_test[0]==4].shape[0] )
print('Link5 samples: ', df_master_master_train[0][df_master_master_train[0]==5].shape[0] +  df_master_master_test[0][df_master_master_test[0]==5].shape[0] )
print('Link6 samples: ', df_master_master_train[0][df_master_master_train[0]==6].shape[0] +  df_master_master_test[0][df_master_master_test[0]==6].shape[0] )
print('Link7 samples: ', df_master_master_train[0][df_master_master_train[0]==7].shape[0] +  df_master_master_test[0][df_master_master_test[0]==7].shape[0] )

number of all samples:  139755
number of noncontact samples:  75809
number of contact samples:  63946
Link1 samples:  2063
Link2 samples:  6403
Link3 samples:  5833
Link4 samples:  9082
Link5 samples:  14648
Link6 samples:  16789
Link7 samples:  9128
